### Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import sys

from math import isnan
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill
from openpyxl.utils import get_column_letter

In [2]:
def crear_ajustar_dataframe(ruta):
    try:
        # Crear dataframe
        df = pd.read_excel(ruta)

        # Ajustar número de columnas del dataframe
        if df.shape[1] > 8:
            df.drop(df.columns[8:], axis=1, inplace=True)
            #print("Bien: Se ajustó el número de columnas.")
            return df
        elif df.shape[1] < 8:
            raise RuntimeError(f"ERROR PASO 1: Número de columnas menor al esperado ({df.shape[1]} < 8).")
        elif df.shape[1] == 8:
            #print("Bien: Número de columnas esperado")
            return df
    
    except FileNotFoundError:
        raise RuntimeError(f"La ruta del archivo XLSX no existe.")
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 1: {e}") from e

In [3]:
def obtener_indice_fila_encabezados(df):
    lista_encabezados = ["Producto", "Nombre producto", "Costeo", "U. base", "U. ref", 
                         "Costo unitario", "Saldo", "Costo total"]
    lista_idx_fila = []

    for idx, encabezado in enumerate(lista_encabezados):
        try:
            # Nombre de columna de dataframe con índice idx
            columna = df.columns[idx]
            # Devuelve una lista vacía o con elementos de los índices coincidentes
            posicion = df[df.loc[:, columna] == encabezado].index[0] 
            # Agregar posición a lista
            lista_idx_fila.append(posicion)
        except IndexError:
            raise RuntimeError(f"ERROR PASO 2: Falta la columna {idx} en el archivo.")
        except ValueError:
            raise RuntimeError(f"ERROR PASO 2: El encabezado '{encabezado}' no se encontró en la columna '{columna}'.")
        except Exception as e:
            raise RuntimeError(f"ERROR PASO 2: {e}") from e
    
    # Ver si la lista tiene como elementos el mismo valor
    if len(set(lista_idx_fila)) == 1:
        #print(f"La fila de encabezados tiene índice {lista_idx_fila[0]}")
        return lista_idx_fila[0]
    else:
        raise RuntimeError("ERROR PASO 2: Los encabezados no están en la misma fila. Revisar el archivo fuente.")

In [4]:
# Dividir dataframe en dos: el primero es de la fila 0 hasta una fila antes de la del encabezado; y el segundo es de la fila
# del encabezado hasta el final del dataframe

def division_dataframe(df, indice_fila_encabezados):
    try:
        # Dataframe antes del encabezado
        df_datos = df.loc[:indice_fila_encabezados-1, :] 
        # Dataframe después del encabezado y resetar índice
        df = df.loc[indice_fila_encabezados:, :].reset_index(drop=True)     
        return df_datos, df
    
    except Exception as e:
        raise RuntimeError("ERROR PASO 3: {e}") from e

In [5]:
def procesamiento_df(df):
    try:
        # Usar la primera fila como nombres de columnas y eliminarla de los datos
        df.columns = df.loc[0]
        df = df.drop(index=0).reset_index(drop=True)


        # Eliminar columnas 'Costeo' y U. base'. Sí existen las columnas con este nombre.
        df.drop(["Costeo", "U. ref"], axis=1, inplace=True)


        # Obtención de índices de las filas que tienen "MARCA" (se encuentran en la columna Producto)
        lista_indices_marca = df[df["Producto"]
                             .astype(str) # Convierte NaN y números a strings 
                             .str.strip() # Quitar espacios al inicio y final
                             .str.upper() # Cambiar cadenas a mayúscula
                             .str.startswith("MARCA", na=False) # Seleccionar filas que empiezan por "MARCA"
                            ].index.tolist()


        # A la fila correspondiente a una MARCA, las columnas distintas a 'Producto' se les asigna un valor np.nan. 
        # A este punto df tiene 6 columnas
        df.iloc[lista_indices_marca, 1:] = np.nan
        
        #-for indice_marca in lista_indices_marca:
        #-    df.loc[indice_marca] = [df.loc[indice_marca]["Producto"]] + [np.nan]*5


        # A las filas donde están Total marca, linea y general (se encuentran en la columna Coasto unitario),
        # a la columna "Saldo" se les asigna un valor np.nan
        lista_indices_mlg0 = df[df["Costo unitario"]
                            .astype(str) # Convierte NaN y números a strings 
                            .str.strip() # Quitar espacios al inicio y final
                            .str.lower() # Cambiar cadenas a mayúscula
                            .str.startswith(("total marca", "total linea", "total general"), na=False)
                           ].index.tolist()
        
        df.loc[lista_indices_mlg0, "Saldo"] = np.nan
        #-for idx in lista_indices_mlg0:
        #-    df.loc[idx, "Saldo"] = np.nan
        
        return df
    
    except Exception as e:
        raise RuntimeError("ERROR PASO 4: {e}") from e

In [6]:
udf = pd.DataFrame({"A":[1,2,3,4,5,6,7,8,9], "B":[11,22,33,44,55,66,77,88,99], "C":[4,5,6,5,4,5,6,5,7]})
udf

,A,B,C
0,1,11,4
1,2,22,5
2,3,33,6
3,4,44,5
4,5,55,4
5,6,66,5
6,7,77,6
7,8,88,5
8,9,99,7


In [7]:
udf.iloc[[2, 4, 6, 8], 1:] = np.nan

In [8]:
udf

,A,B,C
0,1,11.0,4.0
1,2,22.0,5.0
2,3,NaN,NaN
3,4,44.0,5.0
4,5,NaN,NaN
5,6,66.0,5.0
6,7,NaN,NaN
7,8,88.0,5.0
8,9,NaN,NaN


In [9]:
udf.loc[[1,2,3,4], "B"]

1    22.0
2     NaN
3    44.0
4     NaN
Name: B, dtype: float64

In [10]:
def filtrar_df(df):
    try:
        # df5 elimina filas que no tienen relevancia contable: filas de puros NaN o con ceros en las últimas 3 cols
        # 1. Valores True donde "Producto" es NaN
        # 2,3,4. Excluye filas donde "Costo unitario" contiene los textos de totales (marca, línea o general)
        # 5. Convierte los valores a números. Si no se puede (p. ej., texto), devuelve NaN.
        #    True si el valor numérico es mayor que cero
        #    Invierte: True si es NaN o ≤ 0 y False en solo caso > 0.
        df5 = df[
            ~df["Producto"].notna() # 1
            & (df["Costo unitario"] != "Total marca:") # 2
            & (df["Costo unitario"] != "Total linea:") # 3
            & (df["Costo unitario"] != "Total general:") # 4
            & ~(pd.to_numeric(df["Costo unitario"], errors="coerce") > 0) # 5, errors="coerce" forza la conversión
        ]

        # df6 elimina filas que tienen las últimas 3 cols en ceros, pero que sí tienen datos en la columna Producto
        # 1. Solo filas donde "Producto" existe (no es NaN)
        # 2. Convierte "Costo unitario" a número, ignorando errores (NaN si no se puede convertir). 
        #    Solo filas donde el valor es exactamente 0. En ese caso un True.
        # 3. Igual, solo filas donde "Saldo" es 0
        # 4. Igual, solo filas donde "Costo total" es 0
        df6 = df[
            df["Producto"].notna() # 1
            & (pd.to_numeric(df["Costo unitario"], errors="coerce") == 0) # 2
            & (pd.to_numeric(df["Saldo"], errors="coerce") == 0) # 3
            & (pd.to_numeric(df["Costo total"], errors="coerce") == 0) # 4
        ]

        # Indices a eliminar 
        lista_indices_ceros_nan = df5.index.tolist() + df6.index.tolist()

        # Actualizar dataframe 'df' 
        df.drop(index=lista_indices_ceros_nan, inplace=True)

        df = df.reset_index(drop=True)
        return df
    
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 5: {e}") from e

In [11]:
def dividir_bloques_marca(df):
    try:
        # Nuevo dataframe final 1 (df_final1). No tiene filas, está vacío, colo tiene los nombres de las columnas
        df_final1 = pd.DataFrame(columns=df.columns)

        # Crear dataframe cuya única fila tiene elementos np.nan
        df_nan = pd.DataFrame([[np.nan] * len(df.columns)], columns=df.columns)


        # Obtención de índices de las filas que tienen "Total marca"
        lista_indices_marca = df[df["Costo unitario"].astype(str).str.strip().str.startswith("Total marca", na=False)].index.tolist()
        # Obtención de índices de las filas que tienen "Total linea"
        lista_indices_linea = df[df["Costo unitario"].astype(str).str.strip().str.startswith("Total linea", na=False)].index.tolist()
        # Obtención de índices de las filas que tienen "Total general"
        lista_indices_general = df[df["Costo unitario"].astype(str).str.strip().str.startswith("Total general", na=False)].index.tolist()


        # Diccionario: las claves son índices (número de fila) y los valores alguna categoría (marca, linea o general)
        dic_indices_mlg = {}

        for idx in lista_indices_marca:
            dic_indices_mlg[idx] = "marca"
        for idx in lista_indices_linea:
            dic_indices_mlg[idx] = "linea"
        for idx in lista_indices_general:
            dic_indices_mlg[idx] = "general"

        # Ordenación del diccionario dic_indices_mlg de acuerdo a la clave (es un número)
        elementos = dic_indices_mlg.items()
        elementos_ordenados = sorted(elementos, key=lambda item: item[0]) # Orden con respecto al número de fila (a la clave)
        dic_indices_mlg_ord = dict(elementos_ordenados)


        # Introducción de una fila np.nan antes de cada "MARCA", a excepción de que antes de "MARCA" haya una "LINEA".
        fila_inicial = 0

        for idx, (clave, valor) in enumerate(dic_indices_mlg_ord.items()):
            if valor == "marca":
                marca_siguiente = list(dic_indices_mlg_ord.items())[idx + 1][1]

                if valor == marca_siguiente:
                    df_final1 = pd.concat([df_final1, df.loc[fila_inicial:clave, :]], ignore_index=True)
                    df_final1 = pd.concat([df_final1, df_nan], ignore_index=True)
                    fila_inicial = clave + 1
                else:
                    continue

            elif valor == "linea":
                marca_siguiente = list(dic_indices_mlg_ord.items())[idx + 1][1]

                if marca_siguiente == "general":
                    continue
                else:
                    df_final1 = pd.concat([df_final1, df.loc[fila_inicial:clave, :]], ignore_index=True)
                    df_final1 = pd.concat([df_final1, df_nan], ignore_index=True)
                    fila_inicial = clave + 1

            elif valor == 'general':
                df_final1 = pd.concat([df_final1, df.loc[fila_inicial:clave, :]], ignore_index=True)

        return df_final1
    
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 6: {e}") from e

In [12]:
def redondear_si_numero(x, decimal):
    try:
        return round(float(x), decimal)
    except (ValueError, TypeError):
        return x  # si no se puede convertir, lo deja igual

In [13]:
def estetica_df_final1(df_final1):
    try:
        # Redondear las cantidades numéricas de las columnas "Costo unitario", "Saldo" y "Costo total" 
        # a 2, 3 y 2 decimales, respectivamente
        df_final1["Costo unitario"] = df_final1["Costo unitario"].apply(
            lambda x: redondear_si_numero(x, 2)  # aquí defines el número de decimales
        )
        df_final1["Saldo"] = df_final1["Saldo"].apply(
            lambda x: redondear_si_numero(x, 3)
        )
        df_final1["Costo total"] = df_final1["Costo total"].apply(
            lambda x: redondear_si_numero(x, 2) 
        )
        
        # Agregar en la primera fila de df_final1 una fila con los nombres de las columnas
        df_final1 = pd.concat([pd.DataFrame([df_final1.columns], columns=df_final1.columns), df_final1], ignore_index=True)
        return df_final1
    
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 7: {e}") from e

In [14]:
def procesamiento_df_datos(df_datos, df_final1):
    lista_filas_a_guardar = ["GRUPO", "Saldos", "Inventario"]
    lista_indices_filas = []
    try: 
        # Obtener filas que en "Unnamed: 0" empiezan con alguna de las cadenas en lista_filas_a_guardar
        for ele in lista_filas_a_guardar:
            n = df_datos["Unnamed: 0"].fillna("").str.strip().str.startswith(ele).sum()

            if n > 0:
                lista_indices_filas.append(df_datos[df_datos["Unnamed: 0"].fillna("").str.strip().str.startswith(ele)].index[0])
            else:
                raise RuntimeError(f"ERROR PASO 8: No de encuentra fila que inicie por {ele}.")
        
        # Filtrar dataframe por las filas con los índices relevantes y asignar a df_final2
        df_final2 = df_datos.loc[lista_indices_filas]
        # Borrar última y penútlima columnas
        df_final2 = df_final2.drop(df_final2.columns[[-1,-2]], axis=1)
        # Renombrar nombre de columnas como las del dataframe df_final1
        df_final2.columns = df_final1.columns
        
        return df_final2
        
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 8: {e}") from e

In [15]:
def union_df_final_1_2(df_final1, df_final2):
    try:
        return pd.concat([df_final2, df_final1], ignore_index=True)
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 9: {e}") from e

In [16]:
def modificar_aspecto_final(df_final, ruta_archivo_salida):
    try:
        with pd.ExcelWriter(ruta_archivo_salida, engine="openpyxl") as writer:
            df_final.to_excel(writer, index=False, header=False)  # guardar sin encabezados

            # Obtener workbook y worksheet
            wb = writer.book
            ws = writer.sheets["Sheet1"]

            # --- Paso 1 corregido: combinar celdas y centrar en el rango A1:F3 ---
            for row_idx in range(1, 4):  # filas 1, 2, 3
                ws.merge_cells(start_row=row_idx, start_column=1, end_row=row_idx, end_column=6)
                cell = ws.cell(row=row_idx, column=1)  # la celda superior izquierda del rango combinado
                cell.alignment = Alignment(horizontal="center", vertical="center")

            # --- Paso 2: Cuarta fila centrada + relleno gris claro ---
            for cell in ws[4]:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")

            # --- Paso 3: Ajustar ancho de columnas al contenido más largo ---
            for col in ws.columns:
                max_length = 0
                col_letter = get_column_letter(col[0].column)
                for cell in col:
                    try:
                        if cell.value:
                            max_length = max(max_length, len(str(cell.value)))
                    except:
                        pass
                # +2 para dar un poco de espacio
                ws.column_dimensions[col_letter].width = max_length + 8

            # --- Paso 4: Últimas 3 columnas con formato numérico ---
            # Ignorar texto, convertir a float y redondear según columna
            for row in ws.iter_rows(min_row=5, min_col=4, max_col=6):  # columnas 4,5,6 son las últimas 3
                for i, cell in enumerate(row):
                    try:
                        val = float(cell.value)
                        if i == 0:   # primera numérica (col 4)
                            cell.value = round(val, 2)
                        elif i == 1: # segunda numérica (col 5)
                            cell.value = round(val, 3)
                        elif i == 2: # tercera numérica (col 6)
                            cell.value = round(val, 2)
                    except:
                        # Si no es convertible a número, lo dejamos como está
                        pass
    
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 10: {e}") from e

In [17]:
############################################--------------------------------------############################################
def main(ruta_entrada, ruta_salida):
    
    ######### Paso 1. Crear y ajustar dataframe #########
    try:
        df = crear_ajustar_dataframe(ruta=ruta_entrada)
        print("Paso 1 completado.")
    
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1) # Detiene el programa
    
    ######### Paso 2. Obtener índice fila para encabezado #########
    try:
        idx_fila = obtener_indice_fila_encabezados(df=df)
        print("Paso 2 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1) 
        
    ######### Paso 3. #########
    try:
        df_datos, df = division_dataframe(df=df, indice_fila_encabezados=idx_fila)
        print("Paso 3 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
        
    ######### Paso 4. #########
    try:
        df = procesamiento_df(df=df)
        print("Paso 4 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
        
    
    ######### Paso 5. #########
    try:
        df = filtrar_df(df=df)
        print("Paso 5 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
    
    ######### Paso 6. #########
    try:
        df_final1 = dividir_bloques_marca(df=df)
        print("Paso 6 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
    
    ######### Paso 7. #########
    try:
        df_final1 = estetica_df_final1(df_final1)
        print("Paso 7 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
    
    ######### Paso 8.  ######### 
    try:
        df_final2 = procesamiento_df_datos(df_datos=df_datos, df_final1=df_final1)
        print("Paso 8 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
        
    ######### Paso 9.  #########
    try:
        df_final = union_df_final_1_2(df_final1=df_final1, df_final2=df_final2)
        print("Paso 9 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
    
    ######### Paso 10.  #########
    try:
        modificar_aspecto_final(df_final=df_final, ruta_archivo_salida=ruta_salida)
        print("Paso 10 completado.")
    except RuntimeError as err:
        print(f"\n Se produjo un error en el flujo principal:")
        print(f"{err}")
        sys.exit(1)
    
    print("Limpieza del archivo xlsx realizada con éxito.")

In [18]:
# Ruta archivo xlsx original
nombre_archivo = "INVGRAL300925"
ruta_entrada = f".\\{nombre_archivo}.xlsx"
ruta_salida = f".\\{nombre_archivo} filtrado.xlsx"

main(ruta_entrada=ruta_entrada, ruta_salida=ruta_salida)

Paso 1 completado.
Paso 2 completado.
Paso 3 completado.
Paso 4 completado.
Paso 5 completado.
Paso 6 completado.
Paso 7 completado.
Paso 8 completado.
Paso 9 completado.
Paso 10 completado.
Limpieza del archivo xlsx realizada con éxito.
